In [1]:
# notebooks/05_Projektionen.ipynb

import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur erstellen
fig = go.FigureWidget()

# Slider für 3D Vektor
x_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Z:')

# Projektionsoptionen
projection_type = widgets.Dropdown(
    options=['XY-Ebene', 'XZ-Ebene', 'YZ-Ebene'],
    value='XY-Ebene',
    description='Projektion auf:'
)

def update_projection(*args):
    v = np.array([x_slider.value, y_slider.value, z_slider.value])
    
    with fig.batch_update():
        fig.data = []  # Reset data
        
        # Original Vektor (3D)
        fig.add_scatter3d(
            x=[0, v[0]], y=[0, v[1]], z=[0, v[2]],
            mode='lines',
            line=dict(color='blue', width=4),
            name='Original Vektor'
        )
        
        # Projektion
        if projection_type.value == 'XY-Ebene':
            proj = [v[0], v[1], 0]
            proj_line = [v[0], v[1], v[2]]
        elif projection_type.value == 'XZ-Ebene':
            proj = [v[0], 0, v[2]]
            proj_line = [v[0], v[1], v[2]]
        else:  # YZ-Ebene
            proj = [0, v[1], v[2]]
            proj_line = [v[0], v[1], v[2]]
        
        # Projektionsvektor
        fig.add_scatter3d(
            x=[0, proj[0]], y=[0, proj[1]], z=[0, proj[2]],
            mode='lines',
            line=dict(color='red', width=4),
            name='Projektion'
        )
        
        # Projektionslinie
        fig.add_scatter3d(
            x=[proj[0], proj_line[0]], 
            y=[proj[1], proj_line[1]], 
            z=[proj[2], proj_line[2]],
            mode='lines',
            line=dict(color='green', dash='dash'),
            name='Projektionslinie'
        )
        
        # Koordinatensystem
        axis_length = 10
        for i, (color, name) in enumerate([('red', 'X'), ('green', 'Y'), ('blue', 'Z')]):
            fig.add_scatter3d(
                x=[-axis_length, axis_length] if i==0 else [0,0],
                y=[0,0] if i==0 else [-axis_length, axis_length] if i==1 else [0,0],
                z=[0,0] if i<2 else [-axis_length, axis_length],
                mode='lines',
                line=dict(color=color, width=2),
                name=f'{name}-Achse'
            )
        
        # Projektionsebene visualisieren
        if projection_type.value == 'XY-Ebene':
            x_grid, y_grid = np.meshgrid([-10, 10], [-10, 10])
            z_grid = np.zeros_like(x_grid)
        elif projection_type.value == 'XZ-Ebene':
            x_grid, z_grid = np.meshgrid([-10, 10], [-10, 10])
            y_grid = np.zeros_like(x_grid)
        else:  # YZ-Ebene
            y_grid, z_grid = np.meshgrid([-10, 10], [-10, 10])
            x_grid = np.zeros_like(y_grid)
            
        fig.add_surface(
            x=x_grid, y=y_grid, z=z_grid,
            opacity=0.2,
            showscale=False,
            name='Projektionsebene'
        )
        
        # Layout aktualisieren
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-10, 10], showbackground=False),
                yaxis=dict(range=[-10, 10], showbackground=False),
                zaxis=dict(range=[-10, 10], showbackground=False),
                aspectmode='cube'
            ),
            width=800,
            height=800
        )

# Event Handler
for w in [x_slider, y_slider, z_slider, projection_type]:
    w.observe(update_projection, names='value')

# Layout
container = widgets.VBox([
    widgets.HBox([x_slider, y_slider, z_slider]),
    projection_type,
    fig
])

display(container)
update_projection()